In [1]:
import statsmodels.api as sm
import pandas as pd
from sklearn.metrics import mean_squared_error
from utils import get_significant_variables
from lags import create_df_with_lags

In [2]:
df = pd.read_csv("../processed_tables/merged_standardized.csv")

## Creación de lags para el precio de la energía

In [3]:
df = create_df_with_lags(df, 3)
lag_columns = df.filter(like="lag_")
price = df[["Date", "energy_price"]]
df = pd.concat([price, lag_columns], axis=1)
df.head()

,Date,energy_price,energy_price_lag_1,energy_price_lag_2,energy_price_lag_3
3,2021-10-04,286.526116,282.065065,256.940174,216.929726
4,2021-10-05,278.157325,286.526116,282.065065,256.940174
5,2021-10-06,261.474788,278.157325,286.526116,282.065065
6,2021-10-07,235.857848,261.474788,278.157325,286.526116
7,2021-10-08,229.313699,235.857848,261.474788,278.157325


## Partición de la data

Debido a que la naturaleza de la data es de carácter temporal, se realizará una partición de la data en 3 partes: entrenamiento y prueba. La data de test será los últimos 3 meses de la data, es decir desde julio 2023 hasta septiembre 2023.


In [4]:
fecha_corte = "2023-07-01"
X_train = df[df["Date"] < fecha_corte].drop(["Date", "energy_price"], axis=1)
X_test = df[df["Date"] >= fecha_corte].drop(["Date", "energy_price"], axis=1)
y_train = df[df["Date"] < fecha_corte]["energy_price"]
y_test = df[df["Date"] >= fecha_corte]["energy_price"]

## Regresión lineal

Usando `statsmodels` se puede realizar una regresión lineal, para ello se debe importar la librería y crear un modelo de regresión lineal, para luego ajustarlo con los datos de entrenamiento.


In [5]:
X = sm.add_constant(X_train)

# Create a model
model = sm.OLS(y_train, X)

# Fit the model
result = model.fit()
result_summary = result.summary()

# Print out the statistics
print(result_summary)

                            OLS Regression Results                            
Dep. Variable:           energy_price   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     3129.
Date:                Sat, 25 Nov 2023   Prob (F-statistic):               0.00
Time:                        16:06:16   Log-Likelihood:                -2822.3
No. Observations:                 549   AIC:                             5653.
Df Residuals:                     545   BIC:                             5670.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 10.3147      3

In [6]:
get_significant_variables(result_summary)

  Variables significativas
0                    const
1       energy_price_lag_1
2       energy_price_lag_3


Con esto verificamos que el valor de $R^2$ es de $0.514$, lo cual quiere decir que hay una correlación baja entre las variables exógenas y la variable endógena. Las variables significativas son 15 y están asociados a las temperaturas y precipitaciones de ciertos departamentos y también a la temperatura y precipitación de los departamentos que tienen represas.

In [7]:
X_new_with_intercept = sm.add_constant(X_test)
y_pred = result.predict(X_new_with_intercept)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")


MSE: 4537.096764793262
